In [ ]:
def print_versions():
  import pkg_resources

  packages = ["lambeq","discopy","qiskit","qiskit-aer","qiskit-aer-gpu","qiskit-aer-gpu-cu11","pytket","pytket-qiskit","custatevec-cu12","cutensornet-cu12","cuquantum-python-cu12"]


  for package in packages:
      try:
          version = pkg_resources.get_distribution(package).version
          print(f"{package}: {version}")
      except pkg_resources.DistributionNotFound:
          print(f"{package}: Not installed")
      except Exception as e:
          print(f"Error: {e}")

print_versions()

In [1]:
import os
import re
import sys
import json
import pickle
import contextlib
import numpy as np
# from tqdm import tqdm
from typing import List, Tuple, Dict

from qiskit_aer import AerSimulator
# from pytket.extensions.qiskit import AerBackend # toks import veiks su qiskit v2.x


from lambeq.backend.grammar import Diagram, Id
from lambeq import ( BobcatParser, AtomicType, IQPAnsatz, Dataset,
                RemoveCupsRewriter, SimpleRewriteRule, Rewriter, UnifyCodomainRewriter,
                QuantumTrainer, TketModel, BinaryCrossEntropyLoss, SPSAOptimizer
                )


from tensorboard.backend.event_processing import event_accumulator

/home/green/QNLPModelTraining/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Check Qiskit-Aer GPU support
print(AerSimulator().available_devices()) 

AerSimulator().available_methods()


('CPU',)


('automatic',
 'statevector',
 'density_matrix',
 'stabilizer',
 'matrix_product_state',
 'extended_stabilizer',
 'unitary',
 'superop')

In [ ]:
from qiskit_aer import AerSimulator
from pytket.extensions.qiskit import AerBackend

# 1. Patikriname, ar qiskit_aer mato GPU (turi rodyti 'GPU')
print("Qiskit Aer įrenginiai:", AerSimulator().available_devices())

# 2. Sukuriame pytket backend'ą su GPU palaikymu
backend = AerBackend()
# rankiniu būdu nustatome qiskit simuliatoriaus parinktis per pytket
backend._qiskit_backend.set_options(device='GPU')

print(f"Pytket dabar naudoja: {backend._qiskit_backend.options.get('device')}")

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile

# Create a big circuit to make the GPU work
qc = QuantumCircuit(29)
qc.h(range(20))
qc.measure_all()

sim = AerSimulator(method="statevector", device="GPU")
sim = AerSimulator(method="statevector", device="CPU")
available_devices = sim.available_devices()
print(available_devices)
# This will throw an error if the GPU link is "fake"
job = sim.run(transpile(qc, sim), shots=10)
print("Simulation result successful:", job.result().get_counts())

In [ ]:
# 2. Check cuQuantum / cuStateVec
import cuquantum

try:
    print(f"✅ cuQuantum (cuStateVec) Version: {cuquantum.__version__}")
except Exception as e:
    print(f"❌ cuQuantum link error: {e}")

In [ ]:
# green@DESKTOP-5DCJBRK:/downloads$ export PATH=$PATH:/usr/local/cuda-13.1/bin
# green@DESKTOP-5DCJBRK:/downloads$ export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-13.1/lib64

In [ ]:
# custatevec-cu12
# cutensornet-cu12
# cuquantum-python-cu12

In [ ]:
# Compatable version
# "qiskit==1.4.5" "pytket-qiskit==0.66.0" "qiskit-ibm-runtime-0.44.0"

# pip install "qiskit==1.4.5" "pytket-qiskit==0.66.0" qiskit-aer-gpu lambeq discopy tensorboard

# Installed packages:
# Successfully installed MarkupSafe-3.0.3 PyJWT-2.10.1 absl-py-2.3.1 annotated-types-0.7.0 blis-1.3.3 catalogue-2.0.10 certifi-2026.1.4 cffi-2.0.0 charset_normalizer-3.4.4 click-8.3.1 cloudpathlib-0.23.0 confection-0.1.5 contourpy-1.3.3 cryptography-46.0.3 cudensitymat-cu12-0.3.2 cupauliprop-cu12-0.1.0 cuquantum-cu12-25.11.1 custabilizer-cu12-0.1.0 custatevec-cu12-1.11.0 cutensor-cu12-2.4.1 cutensornet-cu12-2.10.1 cycler-0.12.1 cymem-2.0.13 dill-0.4.0 discopy-1.2.2 filelock-3.20.3 fonttools-4.61.1 fsspec-2026.1.0 graphviz-0.21 grpcio-1.76.0 h5py-3.15.1 hf-xet-1.2.0 huggingface-hub-0.36.0 ibm-platform-services-0.73.0 ibm_cloud_sdk_core-3.24.2 idna-3.11 jinja2-3.1.6 kiwisolver-1.4.9 lambeq-0.5.0 lark-1.3.1 markdown-3.10 matplotlib-3.10.8 mpmath-1.3.0 murmurhash-1.0.15 networkx-3.6.1 numpy-2.4.1 nvidia-cublas-cu12-12.8.4.1 nvidia-cuda-cupti-cu12-12.8.90 nvidia-cuda-nvrtc-cu12-12.8.93 nvidia-cuda-runtime-cu12-12.8.90 nvidia-cudnn-cu12-9.10.2.21 nvidia-cufft-cu12-11.3.3.83 nvidia-cufile-cu12-1.13.1.3 nvidia-curand-cu12-10.3.9.90 nvidia-cusolver-cu12-11.7.3.90 nvidia-cusparse-cu12-12.5.8.93 nvidia-cusparselt-cu12-0.7.1 nvidia-nccl-cu12-2.27.5 nvidia-nvjitlink-cu12-12.8.93 nvidia-nvshmem-cu12-3.3.20 nvidia-nvtx-cu12-12.8.90 opt-einsum-3.4.0 pillow-12.1.0 preshed-3.0.12 protobuf-6.33.4 pycparser-2.23 pydantic-2.12.5 pydantic-core-2.41.5 pyparsing-3.3.1 pyspnego-0.12.0 pytket-2.11.0 pytket-qiskit-0.66.0 pyyaml-6.0.3 qiskit-1.4.5 qiskit-aer-0.17.2 qiskit-aer-gpu-0.15.1 qiskit-ibm-runtime-0.44.0 qwasm-1.0.1 regex-2026.1.14 requests-2.32.5 requests-ntlm-1.3.0 rustworkx-0.17.1 safetensors-0.7.0 scipy-1.17.0 setuptools-80.9.0 smart-open-7.5.0 spacy-3.8.11 spacy-legacy-3.0.12 spacy-loggers-1.0.5 srsly-2.5.2 stevedore-5.6.0 symengine-0.13.0 sympy-1.14.0 tensorboard-2.20.0 tensorboard-data-server-0.7.2 tensornetwork-0.4.6 thinc-8.3.10 tokenizers-0.22.2 torch-2.9.1 tqdm-4.67.1 transformers-4.57.5 triton-3.5.1 typer-slim-0.21.1 typing-extensions-4.15.0 typing-inspection-0.4.2 urllib3-2.6.3 wasabi-1.1.3 weasel-0.4.3 werkzeug-3.1.5 wrapt-2.0.1